In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [3]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [4]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [10]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

Success!


In [11]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCostsEdited.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [12]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [13]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

In [14]:
# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

In [15]:
# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Material_Cost'] = 0

In [16]:
# This dataFrame will hold cost issues for all the imaginary builds
missingCost = pd.DataFrame()

In [17]:
""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Material_Cost', -1*(rawGoodsDF['TotalCost'].sum()))
    # missingCost records parts that have 0 cost or are completely missing cost
    noCostDF = rawGoodsDF[(rawGoodsDF['TotalCost'].isnull()) | (rawGoodsDF['TotalCost'] == 0)].copy() # ... it's so nice ...
    noCostDF['Build'] = newbuilds.get_value(index, 'Part')
    missingCost = missingCost.copy().append(noCostDF.copy())

In [18]:
# Create a column representing cost in material either in inventory currently or already on order
newbuilds['Current_Cost_Used'] = newbuilds['Material_Cost'] - newbuilds['Total_Purchase']

In [19]:
# Generic labor estimate based solely on total material cost of the build
newbuilds['Labor_Estimate'] = newbuilds['Material_Cost'] * 0.2

In [20]:
# Cost including materials and labor estimate
newbuilds['Total_Cost'] = newbuilds['Material_Cost'] + newbuilds['Labor_Estimate']

In [21]:
timeline.to_clipboard()

In [22]:
newbuilds.to_clipboard()

In [23]:
missingCost.to_clipboard()

In [ ]:
newbuilds

In [ ]:
missingCost